In [1]:
import os
import json
import google.generativeai as genai
from dotenv import load_dotenv
# from fastapi import FastAPI, HTTPException
# from pydantic import BaseModel
# from typing import List, Optional, Dict
import pandas as pd
from datetime import datetime
import os
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import os
import PyPDF2
import pandas as pd
import numpy as np
from pathlib import Path
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import os
import json

In [2]:
chat_history=[]

In [3]:
def extract_pdf_content(pdf_path):
    """Extract text and content from a single PDF file"""
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            pdf_content = {
                'filename': os.path.basename(pdf_path),
                'total_pages': len(pdf_reader.pages),
                'content': []
            }
            
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                content = {
                    'page_number': page_num + 1,
                    'text': page.extract_text()
                }
                pdf_content['content'].append(content)
            
            print(f"Successfully processed: {pdf_path}")
            return pdf_content
            
    except Exception as e:
        print(f"Error processing {pdf_path}: {str(e)}")
        return None

def process_directory(directory_path):
    pdf_files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]
    
    all_pdf_contents = []
    
    print(f"Found {len(pdf_files)} PDF files")
    
    for pdf_file in pdf_files:
        file_path = os.path.join(directory_path, pdf_file)
        content = extract_pdf_content(file_path)
        if content:
            all_pdf_contents.append(content)
    
    return all_pdf_contents

In [4]:
def combine_pdf_contents(pdf_contents):
    """Combine all PDF contents into a structured format"""
    combined_text = ""
    combined_data = {
        'documents': [],
        'total_pages': 0,
        'extraction_summary': []
    }
    
    for doc in pdf_contents:
        doc_info = {
            'filename': doc['filename'],
            'pages': doc['total_pages']
        }
        combined_data['documents'].append(doc_info)
        combined_data['total_pages'] += doc['total_pages']
        
        combined_text += f"\n\n=== Document: {doc['filename']} ===\n"
        
        for page in doc['content']:
            combined_text += f"\n--- Page {page['page_number']} ---\n"
            combined_text += page['text']
            
        combined_data['extraction_summary'].append({
            'filename': doc['filename'],
            'pages_extracted': doc['total_pages'],
            'has_content': bool(doc['content'])
        })
    
    return combined_text, combined_data

def save_combined_content(combined_text, combined_data, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    text_file_path = os.path.join(output_dir, 'combined_solar_docs.txt')
    with open(text_file_path, 'w', encoding='utf-8') as f:
        f.write(combined_text)
    
    json_file_path = os.path.join(output_dir, 'solar_docs_metadata.json')
    pd.DataFrame(combined_data['documents']).to_json(json_file_path, orient='records', indent=2)
    
    csv_file_path = os.path.join(output_dir, 'extraction_summary.csv')
    pd.DataFrame(combined_data['extraction_summary']).to_csv(csv_file_path, index=False)
    
    return {
        'text_file': text_file_path,
        'json_file': json_file_path,
        'csv_file': csv_file_path
    }

In [5]:
input_dir = "C:\projects\solar_ai_assistance" 
output_dir = "processed_solar_docs" 

print("Starting PDF processing...")
pdf_contents = process_directory(input_dir)

print("\nCombining contents...")
combined_text, combined_data = combine_pdf_contents(pdf_contents)

print("\nSaving combined content...")
saved_files = save_combined_content(combined_text, combined_data, output_dir)

print("\nProcessing complete!")
print("Files saved:")
for file_type, file_path in saved_files.items():
    print(f"{file_type}: {file_path}")

print("\nProcessing Summary:")
print(f"Total documents processed: {len(pdf_contents)}")
print(f"Total pages processed: {combined_data['total_pages']}")

Starting PDF processing...
Found 6 PDF files
Successfully processed: C:\projects\solar_ai_assistance\cost$roi.pdf
Successfully processed: C:\projects\solar_ai_assistance\installation.pdf
Successfully processed: C:\projects\solar_ai_assistance\maintanence.pdf
Successfully processed: C:\projects\solar_ai_assistance\market_trends.pdf
Successfully processed: C:\projects\solar_ai_assistance\production.pdf
Successfully processed: C:\projects\solar_ai_assistance\SCENERIO.pdf

Combining contents...

Saving combined content...

Processing complete!
Files saved:
text_file: processed_solar_docs\combined_solar_docs.txt
json_file: processed_solar_docs\solar_docs_metadata.json
csv_file: processed_solar_docs\extraction_summary.csv

Processing Summary:
Total documents processed: 6
Total pages processed: 362


In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'}
)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4996\2922796233.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [7]:
def load_combined_text(file_path):
    """Load the combined text file"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        print(f"Error loading combined text: {e}")
        return None

def split_text(text):
    """Split text into smaller chunks"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=80,
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    
    chunks = text_splitter.split_text(text)
    print(f"Text split into {len(chunks)} chunks")
    return chunks

def create_metadata(chunks, combined_data):
    """Create metadata for each chunk"""
    metadata = []
    current_doc = None
    current_page = None
    
    for chunk in chunks:
        doc_match = re.search(r"=== Document: (.+?) ===", chunk)
        page_match = re.search(r"--- Page (\d+) ---", chunk)
        
        if doc_match:
            current_doc = doc_match.group(1)
        if page_match:
            current_page = int(page_match.group(1))
            
        metadata.append({
            'document': current_doc,
            'page': current_page,
            'chunk_size': len(chunk)
        })
    
    return metadata

In [8]:
def create_vector_store(chunks, metadata, persist_directory):
    try:
        vectordb = Chroma.from_texts(
            texts=chunks,
            embedding=embeddings,
            metadatas=metadata,
            persist_directory=persist_directory
        )
        
        vectordb.persist()
        print(f"Vector database created and persisted to {persist_directory}")
        return vectordb
        
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return None

def process_and_embed():
    input_file = "processed_solar_docs/combined_solar_docs.txt"
    metadata_file = "processed_solar_docs/solar_docs_metadata.json"
    vector_db_dir = "solar_vector_db"
    
    print("Loading combined text...")
    text = load_combined_text(input_file)
    if text is None:
        return
    
    print("Splitting text into chunks...")
    chunks = split_text(text)
    
    with open(metadata_file, 'r') as f:
        combined_data = json.load(f)
    
    print("Creating metadata...")
    metadata = create_metadata(chunks, combined_data)
    
    print("Creating vector store...")
    vectordb = create_vector_store(chunks, metadata, vector_db_dir)
    
    return vectordb

In [9]:
vectordb = process_and_embed()

Loading combined text...
Splitting text into chunks...
Text split into 2991 chunks
Creating metadata...
Creating vector store...
Vector database created and persisted to solar_vector_db


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4996\759725373.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [10]:
def test_vector_search(vectordb, query, k=10):
    """Test the vector store with a search query"""
    try:
        results = vectordb.similarity_search_with_relevance_scores(
            query=query,
            k=k
        )
        
        print(f"\nSearch results for: '{query}'\n")
        for doc, score in results:
            print(f"Score: {score:.4f}")
            print(f"Document: {doc.metadata.get('document', 'Unknown')}")
            print(f"Page: {doc.metadata.get('page', 'Unknown')}")
            print(f"Content: {doc.page_content[:200]}...")
            print("-" * 80)
            
    except Exception as e:
        print(f"Error during search: {e}")

test_queries = [
    "What is the cost of solar panel installation?",
    "What are the maintenance requirements?",
    "What are the latest market trends in solar energy?"
]

for query in test_queries:
    test_vector_search(vectordb, query)


Search results for: 'What is the cost of solar panel installation?'

Score: 0.5162
Document: maintanence.pdf
Page: 164
Content: --- Page 164 ---
How much can I save after installing a rooftop solar system?
A very common question asked by customers who are keen to install a rooftop solar PV 
system is, “How much will I save aft...
--------------------------------------------------------------------------------
Score: 0.4692
Document: SCENERIO.pdf
Page: 7
Content: [91]. The high installation costs and associated transmission 
and distribution losses of solar power plants pose a significant 
financial challenge. The complexity of plant set-up approvals,...
--------------------------------------------------------------------------------
Score: 0.4590
Document: SCENERIO.pdf
Page: 7
Content: Solar project implementation encounters obstacles in funding, 
land availability for purchase, and solar goods manufacturing, necessitating measures to encourage entrepreneurial activity 
[91]. The hi..